# Práctica 2: Procesamiento de lenguaje natural
## Parte 2: Recuperación de información

##### Grupo Lab: 06 <br> Daniel Alfaro Miranda e Ismail Azizi González 3ºA

### Apartado a) 

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import numpy.ma as ma
import pandas as pd

#Se extraen los datos de entrenamiento y test
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test')

#Se carga el diccionario
with open('words.txt') as f:
    dictionary = f.read().splitlines()

vectorizer = CountVectorizer(vocabulary=dictionary, stop_words='english')

#Se vectorizan los datos con la frecuencia de las palabras
train_vector_data=vectorizer.fit_transform(train_data.data)
test_vector_data=vectorizer.transform(test_data.data)

In [2]:
#Se crea una lista de listas que contiene los indices de las consultas clasificados por su clase
MsgPerClass = [[] for _ in range(len(test_data.target_names))]

for i in range(len(test_data.target)) :
    MsgPerClass[test_data.target[i]].append(i)
    
CorpusPerClass = [[] for _ in range(len(train_data.target_names))]
for i in range(len(train_data.target)) :
    CorpusPerClass[train_data.target[i]].append(i)

In [3]:
def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data:
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')
    
    return terminos.compressed()



In [4]:

#Funcion para calcular similitud del coseno entre consulta y corpus
#Recibe el corpus y la consulta vectorizadas
#Devuelve una lista de tuplas con indice de documento del corpus y su similitud con la consulta,
# ordenadas por similitud decreciente
def getRelatedDocuments(vectorizedCorpus, Query):
    querySim = [(i, cosine_similarity(vectorizedCorpus[i], Query)[0,0]) for i in range(vectorizedCorpus.shape[0])]
    querySim.sort(reverse= True, key= lambda elem : elem[1])
    return querySim

#Funcion que calcula la precision de la lista de resultados para cierto nivel de exhaustividad
#Recibe las clases de los documentos del corpus, la clase de la consulta, la lista de resultados de busqueda y el recall
def resultsPrecision(corpusTargets, targetClass, results, recallLevel):
    relevants = 0
    for i in range(recallLevel):
        if corpusTargets[results[i][0]] == targetClass:
            relevants += 1
    return relevants/recallLevel

#Funcion que calcula valores de precision media en distintos niveles de recall, para n busquedas de una clase de documentos
#Recibe el corpus y las consultas vectorizadas (datos de entrenamiento y test procesados), 
# las clases de los documentos del corpus, los indices del vector de las consultas a evaluar, la clase a evaluar y
# los niveles de recall a calcular
#Devuelve una lista con la precision media para cada nivel de recall
def getClassMeanPrec(vectorizedCorpus, corpusTargets, vectorizedQuerys, selectedQuerysIdx, targetClass, recallLevels):
    means = [0 for _ in range(len(recallLevels))]
    for iQuery in selectedQuerysIdx:
        querySim = getRelatedDocuments(vectorizedCorpus, vectorizedQuerys[iQuery]) #Calculo similitud entre corpus y consulta
        for i, iRlevel in enumerate(recallLevels): #Para cada nivel de recall se calcula la precision del resultado
            means[i] += resultsPrecision(corpusTargets, targetClass, querySim, iRlevel)
    return [i/len(selectedQuerysIdx) for i in means]

#Funcion que devuelve los terminos compartidos entre dos listas
def sharedTerms(msg1, msg2):
    shared = list()
    for term in msg1:
        if term in msg2:
            shared.append(term)
    return shared

#Funcion que devuelve los terminos de la primera lista que no esten en la segunda
def diffTerms(msg1, msg2):
    diff = list()
    for term in msg1:
        if not term in msg2:
            diff.append(term)
    return diff

In [5]:
##Tarda mucho en ejecutar
recallLevels = [3, 10]
querysPerClass = 3
dfData = list()
for i in range(len(test_data.target_names)) :
    means = getClassMeanPrec(train_vector_data, train_data.target, test_vector_data, MsgPerClass[i][:querysPerClass], i, recallLevels)
    dfData.append(means)

cols = ['Precision media con recall ' + str(iLevel) for iLevel in recallLevels]

In [6]:
dfPrec = pd.DataFrame(data=dfData, columns=cols, index=test_data.target_names) 

In [7]:
dfPrec

,Precision media con recall 3,Precision media con recall 10
alt.atheism,0.333333,0.233333
comp.graphics,0.222222,0.200000
comp.os.ms-windows.misc,0.222222,0.300000
comp.sys.ibm.pc.hardware,0.555556,0.500000
comp.sys.mac.hardware,0.000000,0.066667
comp.windows.x,0.333333,0.300000
misc.forsale,0.777778,0.566667
rec.autos,0.333333,0.300000
rec.motorcycles,0.888889,0.733333
rec.sport.baseball,0.777778,0.766667


### Cuestiones:

##### ¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?

Hay bastante diferencia entre distintas clases, por ejemplo rec.motorcycles tiene valores aceptables de precisión en comparación con comp.sys.mac.hardware, que practicamente no acierta en ninguna de sus consultas.

Clases como rec.motorcycles tienen bastante precisión porque sus mensajes cuentan con palabras que difieren mucho de las demás clases. Son mensajes de motos y las demás clases son de política, religión, electronica, etc.
En cambio comp.sys.mac.hardware es muy probable que se este confundiendo con otras clases de temas similares como comp.os.ms-windows.misc o comp.sys.ibm.pc.hardware que no tienen palabras discriminantes entre ellas.

La clase con peores resultados es comp.sys.mac.hardware. Entrando en detalle:

In [8]:
feature_names = vectorizer.get_feature_names()
print("Primera consulta de comp.sys.mac.hardware: ")
print("")
write_terms(feature_names, None, test_vector_data, MsgPerClass[4][0])
print(test_data.data[MsgPerClass[4][0]])

Primera consulta de comp.sys.mac.hardware: 

Mensaje 16 vectorizado: ['boxes' 'college' 'couple' 'different' 'disk' 'documented' 'error' 'file'
 'fills' 'fonts' 'got' 'gradient' 'happen' 'happens' 'help' 'ideas'
 'letters' 'lines' 'organization' 'parts' 'print' 'printing' 'problem'
 'quark' 'subject' 'text' 'thanks' 'things' 'trying' 'uneffected'] 

From: jacobs@cerritos.edu
Subject: Problem printing Quark on a SWII
Organization: Cerritos College, Norwalk CA
Lines: 15

Iv'e got a problem printing with a StyleWriterII. I am printing from a IIvx
with 20 megs ram. I am trying to print a Quark file that has 2 fonts a couple
of boxes and 3 gradient fills. 

Two things happen: I get a " Disk is full" error, that I can't find documented,
I also have parts of letters that are over one of the gradient fills get cut
off. This only happens to the text over the fill. Text adjecent in a different
box is uneffected.

Any ideas?

Thanks for the help...-- 
_____________________________________________

In [10]:
res1 = getRelatedDocuments(train_vector_data, test_vector_data[MsgPerClass[4][0]])
res1

[(10591, 0.3133597930665841),
 (4055, 0.31081147595432446),
 (8216, 0.3090612688845503),
 (5090, 0.2827884099004479),
 (3405, 0.2823419577959951),
 (7264, 0.27911694506082074),
 (4267, 0.2713775413202966),
 (6221, 0.25969228250185833),
 (8156, 0.25777491755679216),
 (8945, 0.25234746934142),
 (7862, 0.24840131369742968),
 (1550, 0.248264415746168),
 (10416, 0.2472490151076403),
 (1567, 0.2467068557457242),
 (693, 0.2466993340805829),
 (7709, 0.2457180467335805),
 (6712, 0.24511108480187255),
 (8785, 0.24244760629816442),
 (6200, 0.2397962597320723),
 (1586, 0.23791547571544328),
 (1090, 0.23791547571544325),
 (7014, 0.2369915286300061),
 (3357, 0.23362835924302428),
 (3453, 0.23362835924302428),
 (4746, 0.23320381162846937),
 (7065, 0.23150776139827395),
 (7654, 0.22893427324781504),
 (3499, 0.22814347892198433),
 (3118, 0.22570643815898364),
 (8315, 0.22348267517713435),
 (6057, 0.22205444400108032),
 (392, 0.2220361701711518),
 (7089, 0.2205887220821391),
 (500, 0.21977690231790253),

In [11]:
for i in range(6):
    print(train_data.target[res1[i][0]])

2
2
2
2
2
2


In [26]:
print("Mensaje de consulta: ")
t1 = write_terms(feature_names, None, test_vector_data, MsgPerClass[4][0])

Mensaje de consulta: 
Mensaje 16 vectorizado: ['boxes' 'college' 'couple' 'different' 'disk' 'documented' 'error' 'file'
 'fills' 'fonts' 'got' 'gradient' 'happen' 'happens' 'help' 'ideas'
 'letters' 'lines' 'organization' 'parts' 'print' 'printing' 'problem'
 'quark' 'subject' 'text' 'thanks' 'things' 'trying' 'uneffected'] 



In [27]:
print("Primer resultado de consulta: ")
t2 = write_terms(feature_names, None, train_vector_data, 10591)

Primer resultado de consulta: 
Mensaje 10591 vectorizado: ['advance' 'answer' 'bought' 'correct' 'disable' 'driver' 'end' 'envelops'
 'feeds' 'got' 'groups' 'history' 'landscape' 'lines' 'macro' 'modify'
 'narrow' 'organization' 'orientation' 'portrait' 'posting' 'present'
 'print' 'printing' 'problem' 'reverse' 'semester' 'subject' 'summarize'
 'thanks' 'wants' 'windows'] 



In [28]:
print("Segundo resultado de consulta: ")
t3 = write_terms(feature_names, None, train_vector_data, 4055)

Segundo resultado de consulta: 
Mensaje 4055 vectorizado: ['ac' 'capacity' 'distribution' 'garbled' 'getting' 'handshaking'
 'laserjet' 'lines' 'missing' 'need' 'newsreader' 'obvious' 'occurred'
 'output' 'posting' 'printing' 'problem' 'serial' 'speed' 'subject' 'sure'
 'suspect' 'thanks' 'tin' 'tweaks' 'used' 'version' 'windows' 'works'
 'world'] 



In [29]:
print("Tercer resultado de consulta: ")
t4 = write_terms(feature_names, None, train_vector_data, 8216)

Tercer resultado de consulta: 
Mensaje 8216 vectorizado: ['away' 'directly' 'holy' 'houses' 'lines' 'option' 'organization'
 'parallel' 'print' 'printing' 'problem' 'setup' 'simular' 'subject'
 'turned' 'went'] 



In [30]:
print("Mensaje de la misma clase que la consulta: ")
t5 = write_terms(feature_names, None, train_vector_data, CorpusPerClass[4][0])

Mensaje de la misma clase que la consulta: 
Mensaje 1 vectorizado: ['acceleration' 'adapters' 'answered' 'article' 'attained' 'brave' 'cards'
 'clock' 'days' 'detailing' 'disk' 'especially' 'experiences' 'final'
 'floppy' 'floppies' 'functionality' 'heat' 'hour' 'keywords' 'knowledge'
 'lines' 'message' 'network' 'number' 'organization' 'oscillator'
 'posting' 'procedure' 'rated' 'reports' 'requested' 'send' 'shared'
 'sinks' 'souls' 'speed' 'subject' 'summary' 'summarizing' 'thanks'
 'upgrade' 'upgraded' 'usage'] 



In [31]:
print("Palabras compartidas: ")
print(sharedTerms(t1, t2))
print('')
print(sharedTerms(t1, t3))
print('')
print(sharedTerms(t1, t4))
print('')
print(sharedTerms(t1, t5))
print('')
print("Palabras diferentes: ")
print(diffTerms(t1, t2))
print('')
print(diffTerms(t1, t3))
print('')
print(diffTerms(t1, t4))
print('')
print(diffTerms(t1, t5))

Palabras compartidas: 
['got', 'lines', 'organization', 'print', 'printing', 'problem', 'subject', 'thanks']

['lines', 'printing', 'problem', 'subject', 'thanks']

['lines', 'organization', 'print', 'printing', 'problem', 'subject']

['disk', 'lines', 'organization', 'subject', 'thanks']

Palabras diferentes: 
['boxes', 'college', 'couple', 'different', 'disk', 'documented', 'error', 'file', 'fills', 'fonts', 'gradient', 'happen', 'happens', 'help', 'ideas', 'letters', 'parts', 'quark', 'text', 'things', 'trying', 'uneffected']

['boxes', 'college', 'couple', 'different', 'disk', 'documented', 'error', 'file', 'fills', 'fonts', 'got', 'gradient', 'happen', 'happens', 'help', 'ideas', 'letters', 'organization', 'parts', 'print', 'quark', 'text', 'things', 'trying', 'uneffected']

['boxes', 'college', 'couple', 'different', 'disk', 'documented', 'error', 'file', 'fills', 'fonts', 'got', 'gradient', 'happen', 'happens', 'help', 'ideas', 'letters', 'parts', 'quark', 'text', 'thanks', 'thi

La gran mayoría de mensajes que recupera para la primera consulta de la clase pertenecen a la clase 2, comp.os.ms-windows.misc. <br>
Los vectores de palabras mostrados corresponden a la consulta, los 3 primeros mensajes recuperados y un mensaje perteneciente a la clase que tendría que devolver. A continuación de estos, se muestran las palabras compartidas y diferentes entre la consulta y estos 4 mensajes. <br>
Como podemos apreciar, los mensajes devueltos por la consulta (comp.os.ms-windows.misc) tienen practicamente las mismas palabras que los mensajes  de la clase de consulta (comp.sys.mac.hardware). <br>
Los malos resutados pueden deberse a que no hay terminos especificos de la clase de consulta que hagan una gran diferencia con la clase 2, podría ser porque los mensajes tratan sobre problemas aplicables tanto a mac como a windows o deberse a que no se han incluido palabras de jerga por el uso de un diccionario.

### Apartado b) 

In [13]:
tfidfer = TfidfTransformer()
train_vectorTfidf_data = tfidfer.fit_transform(train_vector_data)
test_vectorTfidf_data=tfidfer.transform(test_vector_data)

In [32]:
##Tarda mucho en ejecutar
recallLevels = [3, 10]
querysPerClass = 3
dfTfData = list()
for i in range(len(test_data.target_names)) :
    means = getClassMeanPrec(train_vectorTfidf_data, train_data.target, test_vectorTfidf_data, MsgPerClass[i][:querysPerClass], i, recallLevels)
    dfTfData.append(means)

colsTf = ['Precision media con recall ' + str(iLevel) for iLevel in recallLevels]

In [33]:
dfPrecTf = pd.DataFrame(data=dfTfData, columns=colsTf, index=test_data.target_names) 

In [34]:
dfPrecTf

,Precision media con recall 3,Precision media con recall 10
alt.atheism,0.222222,0.266667
comp.graphics,0.111111,0.300000
comp.os.ms-windows.misc,0.111111,0.266667
comp.sys.ibm.pc.hardware,0.666667,0.566667
comp.sys.mac.hardware,0.333333,0.200000
comp.windows.x,0.333333,0.433333
misc.forsale,0.444444,0.400000
rec.autos,0.444444,0.500000
rec.motorcycles,0.777778,0.733333
rec.sport.baseball,1.000000,0.866667


### Cuestiones:

##### ¿Han cambiado los valores de precisión media para las clases del conjunto de datos? ¿Qué clases han mejorado? ¿Cuáles han empeorado?
Han mejorado 9 clases y empeorado 5. En concreto han mejorado las de talk.politics, las de venta y coches (misc.forsale, rec.autos), las de baseball y hockey y las de hardware de pc y mac. Han empeorado la de motos, la de windows.misc, comp.graphics y alt.atheism.

##### Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.
En la clase de rec.sport.hockey	ha mejorado de 0.66 a 1. Vamos a analizar el caso de la primera consulta:

In [14]:
#Consulta con frecuencias
res1 = getRelatedDocuments(train_vector_data, test_vector_data[MsgPerClass[10][0]])
res1

[(721, 0.38729833462074176),
 (3162, 0.38729833462074176),
 (9771, 0.3572172541558802),
 (6510, 0.3396831102433787),
 (8935, 0.3347193406976015),
 (2029, 0.3333333333333334),
 (8484, 0.3333333333333334),
 (3664, 0.32515668876204296),
 (3297, 0.3175264481385602),
 (5944, 0.31622776601683794),
 (9066, 0.31622776601683794),
 (10093, 0.3142696805273545),
 (1813, 0.3118047822311618),
 (3140, 0.3086066999241838),
 (2212, 0.3077287274483319),
 (10483, 0.3061862178478973),
 (2620, 0.29462782549439487),
 (8947, 0.29462782549439487),
 (10714, 0.29462782549439487),
 (3919, 0.29329423004270666),
 (4654, 0.29166666666666674),
 (2540, 0.2886751345948129),
 (3682, 0.2886751345948129),
 (5571, 0.2886751345948129),
 (7853, 0.2886751345948129),
 (10760, 0.28426762180748066),
 (6294, 0.28306925853614895),
 (10090, 0.28306925853614895),
 (1943, 0.2809757434745082),
 (4733, 0.2760262237369417),
 (3244, 0.2738612787525831),
 (6324, 0.2738612787525831),
 (7709, 0.2738612787525831),
 (3006, 0.2727723627949905

In [15]:
for i in range(6):
    print(train_data.target[res1[i][0]])

19
10
2
7
19
9


In [16]:
#consulta con TF/IDF
res2 = getRelatedDocuments(train_vectorTfidf_data, test_vectorTfidf_data[MsgPerClass[10][0]])
res2

[(3664, 0.4590197290975205),
 (3162, 0.36154035065140133),
 (679, 0.3141062782280629),
 (10120, 0.27950845147088643),
 (9319, 0.22245834319715882),
 (5187, 0.21375074252668944),
 (2922, 0.21018481881399323),
 (3919, 0.20483227506690055),
 (5472, 0.19212167586014994),
 (9165, 0.1854147613502106),
 (10332, 0.18378029906162344),
 (4825, 0.17615619401966728),
 (350, 0.1758963835575869),
 (965, 0.17282351106777974),
 (2165, 0.1706595308987447),
 (3769, 0.15851580835025386),
 (7975, 0.15684924717088056),
 (8947, 0.15447354146787784),
 (8533, 0.15163115909744845),
 (8204, 0.15116325950404924),
 (3313, 0.15089810811801918),
 (11050, 0.14778217544682387),
 (5944, 0.1442870723908337),
 (113, 0.14040677782486946),
 (5064, 0.13782839501843752),
 (2409, 0.13702516188990535),
 (5713, 0.1368107842787567),
 (8619, 0.1358353859096964),
 (7084, 0.13464012349808632),
 (1230, 0.13232735964009468),
 (5829, 0.13101177510854756),
 (10978, 0.1307507601845847),
 (10982, 0.1300136873789074),
 (10570, 0.12857158

In [17]:
for i in range(6):
    print(train_data.target[res2[i][0]])

10
10
10
10
10
10


Utilizando frecuencias el primer mensaje es el 721 de la clase talk.religion.misc y utilizando TF/IDF es el 3664 de rec.sport.hockey. Analizando los mensajes: 

In [18]:
print("Mensaje de consulta: ")
t1 = write_terms(feature_names, None, test_vector_data, MsgPerClass[10][0])

Mensaje de consulta: 
Mensaje 56 vectorizado: ['beat' 'caps' 'come' 'cup' 'isles' 'lines' 'make' 'organization'
 'playoffs' 'posting' 'prove' 'ratio' 'said' 'subject' 'watch' 'winning'
 'wouldn' 'wrong'] 



In [19]:
print("Primer resultado usando frecuencias: ")
t2 = write_terms(feature_names, None, train_vector_data, 721)

Primer resultado usando frecuencias: 
Mensaje 721 vectorizado: ['blessed' 'hear' 'yea' 'lines' 'organization' 'posting' 'promise' 'said'
 'software' 'subject'] 



In [20]:
print("Primer resultado usando TF/IDF: ")
t3 = write_terms(feature_names, None, train_vector_data, 3664)

Primer resultado usando TF/IDF: 
Mensaje 3664 vectorizado: ['absolutely' 'al' 'artist' 'astonished' 'attempts' 'bad' 'banjo' 'bench'
 'capital' 'caps' 'centers' 'cheap' 'comes' 'competition' 'continue'
 'couldn' 'crap' 'crowd' 'defensive' 'difficult' 'district' 'ends'
 'figure' 'forwards' 'game' 'games' 'getting' 'goal' 'goals' 'got'
 'gotten' 'hit' 'ice' 'year' 'years' 'inconsistent' 'instead' 'isles'
 'justification' 'later' 'letting' 'level' 'lines' 'maybe' 'make' 'makes'
 'meet' 'minds' 'need' 'neutral' 'opposing' 'opposition' 'organization'
 'penalty' 'penalties' 'period' 'pilon' 'play' 'playoffs' 'police' 'pour'
 'probably' 'remember' 'rest' 'resulting' 'right' 'round' 'sail' 'scored'
 'senseless' 'settling' 'ship' 'shot' 'skater' 'skates' 'software'
 'somebody' 'space' 'stay' 'stupid' 'subject' 'takes' 'talent' 'team'
 'teams' 'think' 'tight' 'tilts' 'tonite' 'turnover' 'understand' 'wish'
 'world' 'worse' 'worst' 'zone'] 



In [21]:
print("Palabras compartidas entre consulta y mensaje de frecuencias: ")
print(sharedTerms(t1, t2))
print('')
print("Palabras diferentes: ")
print(diffTerms(t1, t2))

Palabras compartidas entre consulta y mensaje de frecuencias: 
['lines', 'organization', 'posting', 'said', 'subject']

Palabras diferentes: 
['beat', 'caps', 'come', 'cup', 'isles', 'make', 'playoffs', 'prove', 'ratio', 'watch', 'winning', 'wouldn', 'wrong']


In [22]:
print("Palabras compartidas entre consulta y TF/IDF: ")
print(sharedTerms(t1, t3))
print('')
print("Palabras diferentes: ")
print(diffTerms(t1, t3))

Palabras compartidas entre consulta y TF/IDF: 
['caps', 'isles', 'lines', 'make', 'organization', 'playoffs', 'subject']

Palabras diferentes: 
['beat', 'come', 'cup', 'posting', 'prove', 'ratio', 'said', 'watch', 'winning', 'wouldn', 'wrong']


In [23]:
print("Mensaje de consulta: ")
print(test_data.data[MsgPerClass[10][0]])

Mensaje de consulta: 
From: jgold@chopin.udel.edu (Jonathan Goldstein)
Subject: Re: The Amazin' Isles!!!!!!
Nntp-Posting-Host: chopin.udel.edu
Organization: University of Delaware
Lines: 21

WATCH OUT PITSBURGH HERE COME THE ISLES!!!!!!!!!!!!!!!!!!!!!!!!!



They said we wouldn't make the playoffs and we came in third
They said the Caps would beat us and they're not going to
They say that Pitsburgh has a 1:1 ratio of winning the cup but We'll prove them
wrong.


L E T S   G O    I S L A N D E R S!!!!!!!  Bring it back home





-- 
            //////////   /////////   //      //
                  //       //     //   // /   //
                 //        //     //   //  /  //
                //         //     //   /    / //



Ponderando los pesos por frecuencia el primer resultado que se obtiene es el mensaje 721 de la clase talk.religion.misc.<br>
Si se observan los mensajes y sus terminos, se puede ver que son mensajes cortos que lo que tienen en comun es principalmente la cabecera, con palabras como 'lines', 'organization', etc. Al ser cortos tienen muy pocas diferencias y es el mensaje mas similar a la consulta.
<br>En el caso de TF/IDF estas palabras de las cabeceras tienen mucho menos valor y por lo tanto se valoran más las palabras discriminantes de la clase. Comparando con el primer resultado de esta consulta (mensaje 3664), este pertenece a la misma clase que la consulta por palabras discriminantes como 'isles' o 'caps' que representan equipos de hockey, si se lee el mensaje de consulta.

In [24]:
linesIdx = feature_names.index('lines')
print("Valor en frecuencia: " + str(test_vector_data[MsgPerClass[10][0], linesIdx]))
print("Valor en TF/IDF: " + str(test_vectorTfidf_data[MsgPerClass[10][0], linesIdx]))

Valor en frecuencia: 1
Valor en TF/IDF: 0.04366734062657974


In [25]:
capsIdx = feature_names.index('caps')
print("Valor en frecuencia: " + str(test_vector_data[MsgPerClass[10][0], capsIdx]))
print("Valor en TF/IDF: " + str(test_vectorTfidf_data[MsgPerClass[10][0], capsIdx]))

Valor en frecuencia: 1
Valor en TF/IDF: 0.2596008048124393


Comparando los valores de la palabra 'lines' y 'caps' pasamos del mismo valor en frecuencia a un valor TF/IDF mucho más representativo de la clase.